In [ ]:
import numpy as np
import sys

import LUT_Model_OAB as OAB
import LUT_Model_BOO as BOO
sys.path.append('../') # Move to root directory to match README.md instructions
from model import LUT_Model as control

import warnings
warnings.filterwarnings("ignore")

## Hypothesis: Increased parasympathetic neural input and decreased sympathetic neural input results in a decreased intervoid interval.

In [4]:
seed = 42
dT = 0.1
day = 24 * 60 * 60
maxTime = 3 * day

# Run the simulation
OAB_LUT = OAB.LUT()
OAB_data = OAB_LUT.process_neural_input(maxTime, dT, seed=seed, verbose=True, p_unit="cmH2O", V_unit="ml")

control_LUT = control.LUT()
data_control = control_LUT.process_neural_input(maxTime, dT, seed=seed, verbose=True, p_unit="cmH2O", V_unit="ml")

In [5]:
def get_intervoid_interval(data):
    voiding_indices = np.where(np.diff(data['voiding']) == 1)[0]

    # Intervoid interval - time between two voiding phases
    intervoid_interval = np.diff(data.iloc[voiding_indices]['t'])

    return intervoid_interval

# Get the intervoid interval for the control and OAB data
intervoid_interval_control = get_intervoid_interval(data_control)
intervoid_interval_OAB = get_intervoid_interval(OAB_data)

# Get average intervoid interval
mean_intervoid_interval_control = np.mean(intervoid_interval_control)
mean_intervoid_interval_OAB = np.mean(intervoid_interval_OAB)

print(f"Mean intervoid interval for control: {mean_intervoid_interval_control:.2f} seconds")
print(f"Mean intervoid interval for OAB: {mean_intervoid_interval_OAB:.2f} seconds")
reduction_percentage = (mean_intervoid_interval_control - mean_intervoid_interval_OAB) / mean_intervoid_interval_control * 100
print(f"Reduction in intervoid interval: {reduction_percentage:.2f}%")

Mean intervoid interval for control: 21360.90 seconds
Mean intervoid interval for OAB: 14731.21 seconds
Reduction in intervoid interval: 31.04%


## Hypothesis: Increased parasympathetic neural input and decreased sympathetic neural input results in a decreased maximum bladder capacity.

In [6]:
# Get maximum bladder capacity
max_bladder_capacity_control = np.max(data_control['V_B'])
max_bladder_capacity_OAB = np.max(OAB_data['V_B'])

print(f"Maximum bladder capacity for control: {max_bladder_capacity_control:.2f} ml")
print(f"Maximum bladder capacity for OAB: {max_bladder_capacity_OAB:.2f} ml")
reduction_percentage = (max_bladder_capacity_control - max_bladder_capacity_OAB) / max_bladder_capacity_control * 100
print(f"Reduction in maximum bladder capacity: {reduction_percentage:.2f}%")

Maximum bladder capacity for control: 489.58 ml
Maximum bladder capacity for OAB: 334.89 ml
Reduction in maximum bladder capacity: 31.60%


## Hypothesis: Increasing urethral resistance results in decreased flow rate.

In [7]:
BOO_LUT = BOO.LUT()
BOO_data = BOO_LUT.process_neural_input(maxTime, dT, seed=seed, verbose=True, p_unit="cmH2O", V_unit="ml")

In [8]:
max_flow_rate_BOO = np.max(BOO_data['Q'])
max_flow_rate_control = np.max(data_control['Q'])
print(f"Maximum flow rate for BOO: {max_flow_rate_BOO:.2f} ml/s")
print(f"Maximum flow rate for control: {max_flow_rate_control:.2f} ml/s")
reduction_percentage = (max_flow_rate_control - max_flow_rate_BOO) / max_flow_rate_control * 100
print(f"Reduction in maximum flow rate: {reduction_percentage:.2f}%")

Maximum flow rate for BOO: 12.31 ml/s
Maximum flow rate for control: 21.11 ml/s
Reduction in maximum flow rate: 41.71%
